In [7]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf

In [8]:
df = pd.read_csv('total_data.csv').set_index('Zipcode')

### Double lasso

We lasso regress Covid against all controls, and KDE against all controls.  We then observe which controls have nonzero coefficients in one of the two cases.

In [9]:
lasso_alpha = 0.1
selection_cutoff = 0.1

# min max normalization (ols handles mean)
normalized_df = (df-df.min())/(df.max()-df.min())

In [10]:
DoubleLassoFormula0 = """
    Covid ~ Men + Hispanic + White + Black + Native + Asian + Citizen + Income + IncomeErr + IncomePerCap + IncomePerCapErr + Poverty + ChildPoverty + Professional + Service + Office + Construction + Production + Drive + Carpool + Transit + Walk + OtherTransp + WorkAtHome + MeanCommute + Employed + PrivateWork + PublicWork + SelfEmployed + FamilyWork + Unemployment + Density
"""

lmod0 = smf.ols(formula=DoubleLassoFormula0, data=normalized_df)
lres0 = lmod0.fit_regularized(method='sqrt_lasso', alpha=lasso_alpha, cov_type='HC3')

stage_zero_selected = {
    p:v for p, v in dict(lres0.params).items()
    if np.abs(v) > selection_cutoff
}

DoubleLassoFormula1 = """
    KDE ~ Men + Hispanic + White + Black + Native + Asian + Citizen + Income + IncomeErr + IncomePerCap + IncomePerCapErr + Poverty + ChildPoverty + Professional + Service + Office + Construction + Production + Drive + Carpool + Transit + Walk + OtherTransp + WorkAtHome + MeanCommute + Employed + PrivateWork + PublicWork + SelfEmployed + FamilyWork + Unemployment + Density
"""

lmod1 = smf.ols(formula=DoubleLassoFormula1, data=normalized_df)
lres1 = lmod1.fit_regularized(method='sqrt_lasso', alpha=lasso_alpha, cov_type='HC3')

stage_one_selected = {
    p:v for p, v in dict(lres1.params).items()
    if np.abs(v) > selection_cutoff
}

total_lasso_selected = set(stage_zero_selected.keys()) | set(stage_one_selected.keys())
total_lasso_discarded = set(dict(lres0.params[1:]).keys()).difference(total_lasso_selected)

print(f'Selected {len(total_lasso_selected)}:', *total_lasso_selected)
print(f'Discarded {len(total_lasso_discarded)}:', *total_lasso_discarded)

Selected 23: ChildPoverty OtherTransp Native IncomeErr White IncomePerCapErr Hispanic MeanCommute Construction IncomePerCap PublicWork Citizen Service Men Black Walk Unemployment Drive Employed Income Office Carpool FamilyWork
Discarded 9: Professional SelfEmployed Poverty WorkAtHome Asian Production Transit Density PrivateWork


### Run regression

In [11]:
Formula = "Covid ~ KDE + " + " + ".join(total_lasso_selected)

mod = smf.ols(formula=Formula, data=df)
res = mod.fit()

res.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  Covid   R-squared:                       0.620
Model:                            OLS   Adj. R-squared:                  0.558
Method:                 Least Squares   F-statistic:                     9.995
Date:                Sun, 22 Nov 2020   Prob (F-statistic):           1.47e-20
Time:                        19:36:38   Log-Likelihood:                -199.19
No. Observations:                 172   AIC:                             448.4
Df Residuals:                     147   BIC:                             527.1
Df Model:                          24                                         
Covariance Type:            nonrobust                                         
===================================================================================
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept          -0.3156      1.738     -0.182      0.856      -3.750       3.119
KDE                 0.0032      0.005      0.614      0.540      -0.007       0.013
ChildPoverty        0.0025      0.012      0.209      0.834      -0.021       0.026
OtherTransp        -0.0540      0.055     -0.976      0.330      -0.163       0.055
Native             -0.0766      0.192     -0.399      0.690      -0.456       0.303
IncomeErr        1.535e-05   1.86e-05      0.827      0.409   -2.13e-05     5.2e-05
White               0.0118      0.008      1.450      0.149      -0.004       0.028
IncomePerCapErr  2.796e-05    5.4e-05      0.518      0.605   -7.87e-05       0.000
Hispanic            0.0079      0.009      0.913      0.363      -0.009       0.025
MeanCommute         0.0506      0.024      2.094      0.038       0.003       0.098
Construction       -0.0593      0.039     -1.520      0.131      -0.136       0.018
IncomePerCap     8.708e-06   1.52e-05      0.574      0.567   -2.13e-05    3.87e-05
PublicWork          0.0199      0.023      0.861      0.391      -0.026       0.066
Citizen            -0.0350      0.014     -2.447      0.016      -0.063      -0.007
Service             0.0834      0.024      3.463      0.001       0.036       0.131
Men                 0.0011      0.024      0.046      0.964      -0.046       0.048
Black              -0.0201      0.008     -2.428      0.016      -0.037      -0.004
Walk                0.0184      0.014      1.323      0.188      -0.009       0.046
Unemployment       -0.0182      0.032     -0.561      0.576      -0.082       0.046
Drive               0.0369      0.010      3.882      0.000       0.018       0.056
Employed         -128.8863    107.816     -1.195      0.234    -341.956      84.184
Income          -5.358e-06   9.31e-06     -0.576      0.566   -2.37e-05     1.3e-05
Office              0.0326      0.026      1.235      0.219      -0.020       0.085
Carpool            -0.0830      0.042     -1.994      0.048      -0.165      -0.001
FamilyWork         -0.8580      0.427     -2.010      0.046      -1.702      -0.014
==============================================================================
Omnibus:                        0.073   Durbin-Watson:                   1.945
Prob(Omnibus):                  0.964   Jarque-Bera (JB):                0.041
Skew:                           0.036   Prob(JB):                        0.980
Kurtosis:                       2.975   Cond. No.                     1.47e+08
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.47e+08. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [13]:
print(res.summary().as_latex())

\begin{center}
\begin{tabular}{lclc}
\toprule
\textbf{Dep. Variable:}    &      Covid       & \textbf{  R-squared:         } &     0.620   \\
\textbf{Model:}            &       OLS        & \textbf{  Adj. R-squared:    } &     0.558   \\
\textbf{Method:}           &  Least Squares   & \textbf{  F-statistic:       } &     9.995   \\
\textbf{Date:}             & Sun, 22 Nov 2020 & \textbf{  Prob (F-statistic):} &  1.47e-20   \\
\textbf{Time:}             &     19:37:22     & \textbf{  Log-Likelihood:    } &   -199.19   \\
\textbf{No. Observations:} &         172      & \textbf{  AIC:               } &     448.4   \\
\textbf{Df Residuals:}     &         147      & \textbf{  BIC:               } &     527.1   \\
\textbf{Df Model:}         &          24      & \textbf{                     } &             \\
\bottomrule
\end{tabular}
\begin{tabular}{lcccccc}
                         & \textbf{coef} & \textbf{std err} & \textbf{t} & \textbf{P$> |$t$|$} & \textbf{[0.025} & \textbf{0.975]}  \\
